In [18]:
import jieba
import MySQLdb
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

jieba.set_dictionary('dict.txt.big')
jieba.load_userdict('userdict.txt')

conn = MySQLdb.connect(host="localhost", user="root", passwd="", db="python",charset='utf8')#連結資料庫
cur = conn.cursor() 
cur.execute("SELECT search_href,id FROM mobile01 WHERE content_analyst='-1'")
results = cur.fetchall()


with open('positive.txt', 'r',encoding='UTF-8') as positive:
    pos=[]
    for line in positive:
        pos.append(line.strip('\ufeff').strip())
    positive.close()

with open('nagative.txt', 'r',encoding='UTF-8') as nagative:
    nag=[]
    for line in nagative:
        nag.append(line.strip('\ufeff').strip())
    nagative.close()

with open('paid news.txt', 'r',encoding='UTF-8') as paidnews:
    paid=[]
    for line in paidnews:
        paid.append(line.strip('\ufeff').strip())
    paidnews.close()
    
print(pos)
print(nag)
print(paid)

pos_set=set(pos)
nag_set=set(nag)
paid_set=set(paid)

for record in results: 
    db_url = record[0]
    mobile01_id= record[1]
    print (db_url)
    
    res=requests.get(db_url)
    res.encoding='utf-8'
    soup = BeautifulSoup (res.text, "html5lib")
    main_article=soup.select('.single-post-content')
    content=main_article[0].text.strip()
    sentence=content.split("\n")
    sentence=remove_values_from_list(sentence,'')

    total_pos_count=0
    total_nag_count=0
    total_paid_count=0
    for line in sentence:

        line2=line.strip()
        words=jieba.lcut(line2, cut_all=False)
        words_set = set(words)
        pos_intersection=words_set.intersection(pos_set)
        nag_intersection=words_set.intersection(nag_set)
        paid_intersection=words_set.intersection(paid_set)
        pos_count=len(pos_intersection)
        nag_count=len(nag_intersection)
        paid_count=len(paid_intersection)

        total_pos_count=total_pos_count+pos_count
        total_nag_count=total_nag_count+nag_count
        total_paid_count=total_paid_count+paid_count
    total_count = total_pos_count+total_nag_count+total_paid_count


    if total_count==0:
        Content_Analyst = "0"

    else:
        Content_Analyst = format(total_pos_count/total_count*100 , '0.2f')

    print(Content_Analyst)
    cur.execute ("UPDATE mobile01 SET content_analyst=%s WHERE id='%s'" %  (Content_Analyst,mobile01_id))
    conn.commit()
    
cur.close() #斷開連結
conn.close()


Building prefix dict from C:\Users\wlunareve\python\dict.txt.big ...
Loading model from cache C:\Users\WLUNAR~1\AppData\Local\Temp\jieba.ube0c4e67d291f94013d7023df2b3161e.cache
Loading model cost 1.304 seconds.
Prefix dict has been built succesfully.


['超', '太', '真', '', '超美', '很美', '好美', '太大', '好大', '很大', '超大', '真大', '太好', '真好', '超好', '太便宜', '', '吸睛', '歐耶', '過癮', '齊全', '絕配', '極致', '享受', '滿足', '有感', '力推', '佛心', '浮誇', '心動', '對味', '幸福']
['不大', '不佳', '不優', '一般', '誠實', '老實', '普普', '普通', '不錯', '失望', '不推', '失落', '還行', '瑕疵', '斟酌']
['招待', '工商', '試用', '體驗', '愛評網', '推薦', '邀約', '免費', '分享']
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D602%26t%3D4523311&sa=U&ved=0ahUKEwitmL7u993TAhXMvrwKHWKHBo0QFggTMAA&usg=AFQjCNGwsul2EkNQEueslMYbtPClAQaLxA
0
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D37%26t%3D1128622&sa=U&ved=0ahUKEwitmL7u993TAhXMvrwKHWKHBo0QFggZMAE&usg=AFQjCNHmG2TdA5eo2_L8EDu6rDUd0VyIjQ
100.00
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D602%26t%3D2778831&sa=U&ved=0ahUKEwitmL7u993TAhXMvrwKHWKHBo0QFggfMAI&usg=AFQjCNE5EZVUR3uhmmNBFWWOL3Cf_kh4RQ
100.00
https://www.google.com//url?q=https://www.mobile01.com/waypointtopicdetail.php%3Ff%3D197%26t%3D4996